<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900" style="text-align:center;">**Notebook 6.4:** 语意护栏</font>

<br>

此 notebook 是 **Notebook 6：嵌入模型和语义推理**内容的延伸。但由于其长度及主题相关性，就单独用一个 notebook 来讲解。

<br>  

### **环境设置：**

In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -qq langchain langchain-nvidia-ai-endpoints gradio

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

NVIDIAEmbeddings.get_available_models()

----

<br>

## **第 4 部分：[进阶练习]** 嵌入语义护栏

在下一个 notebook 中，我们将开始用更高级的工具，它们将在背后使用我们的嵌入模型。不过现在，我们仍然可以先用原始的方式探索几个重要的概念！

具体来说，我们可以将其用作生产中部署的模型的一个关键组件：**语义护栏**。可以借助嵌入来过滤掉聊天机器人无法回答的消息（或有害的消息）！

<br>

### **相比自回归的优势**

您可能还记得，我们在之前的 notebook 中使用 LLM 来实现复杂的内部推理，那是不是也能用来过滤？具体来说，您可能想让 LLM 判断问题，然后借助 `RunnableBranch` 切换分支。您当然可以这样来实现，但有一些明显的优缺点需要进一步考虑：

> **优点：**用提示词让内部系统限制对话进程相对来说比较容易。您甚至可以开发一个看过很多好坏问题例子的例程，通过一套提示词稳定地返回“好”或“坏”。
> 
> **缺点：**自回归的方式可能会带来无法接受的延迟或资源开销。例如，您可能希望在背后集成语义护栏机制，防止有害输出，将有问题的输入向安全和可预测的方向引导。用自回归可以这样做：
> 
> * **您可以使用相对较小的指令微调模型作为零样本分类器，期望它能保持性能稳定。**为了做到这一点，您可能还需要将输入转成某种模型擅长应对的标准形式。
> * **您还可以针对任务微调小型自回归 LLM。**您需要合成一些数据，并会花费一些预算用来做一次微调，但至少这能让小一点的模型更接近做了提示工程后的大模型性能。

虽然这些都是可行的方式，但我们还可以通过适当的嵌入模型、合成数据以及深度学习的基本思路来处理这个问题。

**具体来说，我们可以把一个嵌入模型作为语言主干，然后在之上训练分类器。**接下来就实现一下这个思路，看看怎么处理实现中的挑战。

<br>  

#### **任务 1：生成合成数据**

要开始制作语义护栏，我们显然需要先制定一些目标。

* **假设：**假设我们想制作一个 NVIDIA 聊天机器人，主要讨论技术和公司相关的细节。要是您觉得这样的聊天机器人定义存在明显的漏洞，那就对了！但不妨碍把它作为一个有趣的起点，利于我们之后扩展到实际场景中！

* **计划：**为了明确我们面临的问题，最好生成一些有代表性输入，来帮助定义*好的*输入和*差的*输入。然后观察嵌入模型是怎么处理这些例子的，并相应地设计解决方案。

遗憾的是，我们没有任何真实数据，看来必须合成数据了！作为练习，先生成一些有代表性的好的和差的示例。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ChatMessage
from operator import itemgetter

## Useful method for mistral, which is currently tuned to output numbered outputs
def EnumParser(*idxs):
    '''Method that pulls out values from a mistral model that outputs numbered entries'''
    idxs = idxs or [slice(0, None, 1)]
    entry_parser = lambda v: v if ('. ' not in v) else v[v.index('. ')+2:]
    out_lambda = lambda x: [entry_parser(v).strip() for v in x.split("\n")]
    return StrOutputParser() | RunnableLambda(lambda x: itemgetter(*idxs)(out_lambda(x)))

instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1") | EnumParser()

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

gen_prompt = {'input' : lambda x:x} | ChatPromptTemplate.from_template(
    "Please generate 20 representative conversations that would be {input}."
    " Make sure all of the questions are very different in phrasing and content."
    " Do not respond to the questions; just list them. Make sure all of your outputs are numbered."
    " Example Response: 1. <question>\n2. <question>\n3. <question>\n..."
)

## Some that directly reference NVIDIA
responses_1 = (gen_prompt | instruct_llm).invoke(
    " reasonable for an NVIDIA document chatbot to be able to answer."
    " Vary the context to technology, research, deep learning, language modeling, gaming, etc."
)
print("Reasonable NVIDIA Responses:", *responses_1, "", sep="\n")

## And some that do not
responses_2 = (gen_prompt | instruct_llm).invoke(
    " be reasonable for a tech document chatbot to be able to answer. Make sure to vary"
    " the context to technology, research, gaming, language modeling, graphics, etc."
)
print("Reasonable non-NVIDIA Responses:", *responses_2, "", sep="\n")

## Feel free to try your own generations instead
responses_3 = (gen_prompt | instruct_llm).invoke(
    "unreasonable for an NVIDIA document chatbot to answer,"
    " as it is irrelevant and will not be useful to answer (though not inherently harmful)."
)
print("Irrelevant Responses:", *responses_3, "", sep="\n")

responses_4 = (gen_prompt | instruct_llm).invoke(
    "unreasonable for a chatbot (NVIDIA's, AMD's, Intels, or Generally) to answer,"
    " as an automated response will either be overly insensitive or offensive."
)
print("Harmful non-NVIDIA", *responses_4, "", sep="\n")

## Feel free to try your own generations instead

good_responses = responses_1 + responses_2
poor_responses = responses_3 + responses_4

<br>

### **任务 2：生成更多嵌入（更快地）**

当您对合成数据满意后，就该把它们都嵌入到语义向量中了。我们之前用 `embed_query` 和 `embed_documents` 这类同步命令来嵌入文档，对较小或实时数据可能就足够了。但当我们需要一次性执行大量嵌入时，就会遇到瓶颈。

本节我们将使用**异步技术**来允许同时进行多个嵌入！这是一种通常在背后自动运行的机制。**它*无法达到*无限并发**，在手动将其集成到大型部署之前，应进行更深入的研究。

<br>  

#### **计时**

`%%time` 不适用于在 notebook 中监控异步解决方案，我们可以用下面基于 scope 的计时方式。我们在下方的单元中测试了嵌入 10 个文档的用时：

In [ ]:
import time
import numpy as np

class Timer():
    '''Useful timing utilities (%%time is great, but doesn't work for async)'''
    def __enter__(self):
      self.start = time.perf_counter()

    def __exit__(self, *args, **kwargs):
        elapsed = time.perf_counter() - self.start
        print("\033[1m" + f"Executed in {elapsed:0.2f} seconds." + "\033[0m")

with Timer():
    good_embeds = [embedder.embed_query(x) for x in good_responses[:10]]

print("Shape:", np.array(good_embeds).shape)

<br>  

#### **异步嵌入**

您应该注意到了，执行此嵌入花费了大量时间。如果我们拥有对嵌入模型的原始访问权限，就能通过批处理响应（batching responses）轻松地加速。其实，云端的查询路由已经自动执行此操作了，但出于公平和同质性的考虑仅允许每个用户同时提交一次查询。

换言之，不是服务无法加快嵌入速度，而是我们的代码正在等待一系列的 `embed_query` 命令逐个完成。

当我们需要同时嵌入大量文档时，最好同时**异步地**提交所有请求并等待结果返回。如果实现的合理，这将大大加快本地的嵌入过程，同时几乎不对 LLM 服务产生任何影响（假设查询路由应用了[**动态批处理**（in-flight batching）](https://github.com/NVIDIA/TensorRT-LLM/blob/b777bd64750abf30ca7eda48e8b6ba3c5174aafd/docs/source/advanced/gpt-attention.md?plain=1#L137)，这是一种将多个请求以批的形式堆叠着送入神经网络的方式）。

我们可以测试 LangChain 的 `aembed_<...>`，生成一些用于**并发**执行的**协程**（Coroutine）：

In [ ]:
with Timer():
    good_embed_gens = [embedder.aembed_query(query) for query in good_responses[10:20]]
print(good_embed_gens[0])

## NOTE: When you define coroutines, you will want to either execute them or close them.
##  Destroying an open coroutine object by overriding will throw a warning.
for gen in good_embed_gens:
    gen.close()

您可以使用 `await` 关键字或类似于 [`asyncio.gather`](https://docs.python.org/3/library/asyncio-task.html#id8) 的例程来逐个等待它们执行完成。用后一种方式的话，asyncio 将同时执行所有这些协程，并在最后一个协程完成时聚合或者说**收集**响应。

In [ ]:
import asyncio

with Timer():
    tasks = [embedder.aembed_query(query) for query in good_responses[10:20]]
    good_embeds2 = await asyncio.gather(*tasks)

print("Shape:", np.array(good_embeds2).shape)

之前的非异步版本显示了**串联执行**所有这些嵌入需要的时间，现在这个新的时间就反映了并发的耗时，大致就是最长的单个嵌入请求耗时。

<br> 

#### **限制并发性**

虽然此系统的速度明显快于同步版本，但一定要注意并发不能无限地堆叠！如果并发地运行了过多的任务，可能会搞砸，服务可能对您进行节流，或者耗尽资源。在实践中，最好限制最大的并发次数，比如可以借助限制最大并发的异步**信号量**（semaphore）：

In [ ]:
import asyncio
from collections import abc
from typing import Callable
from functools import partial

async def embed_with_semaphore(
    text : str,
    embed_fn : Callable,
    semaphore : asyncio.Semaphore
) -> abc.Coroutine:
    async with semaphore:
        return await embed_fn(text)

## Making new embed method to limiting maximum concurrency
embed = partial(
    embed_with_semaphore,
    embed_fn = embedder.aembed_query,
    semaphore = asyncio.Semaphore(value=10)  ## <- feel free to play with value
)

## This is once again a coroutine constructor, so should take marginal time
tasks = [embed(query) for query in good_responses[20:30]]

with Timer():
    good_embeds_3 = await asyncio.gather(*tasks)

<br>  

#### **[练习] 嵌入剩余的文档**

您已经了解这套流程的做法了，现在就来嵌入剩下的文档吧。尝试将并发控制在合理的范围内（如果失败了，您会知道的），看看您是否能让它足够快。

经过测试，我们发现 10 是个最优的值，超过之后并发的优势就开始下降了。在选择值时请记住这一点。

In [ ]:
####################################################################################
## BEGIN TODO

## Note, we found marginal benefit after value=10 in our tests...
with Timer():
    good_tasks = []
    poor_tasks = []
    good_embeds = []
    poor_embeds = []

print("Good Embeds Shape:", np.array(good_embeds).shape)
print("Poor Embeds Shape:", np.array(poor_embeds).shape)

## END TODO
####################################################################################

### **任务 3：确认语义密度**

我们生成这些嵌入依赖于它们会对语意过滤有帮助的假设。为了确认这一点，我们可以使用一些经典的机器学习方法，例如[**主成分分析**（PCA）](https://en.wikipedia.org/wiki/Principal_component_analysis)或 [t-分布式随机近邻嵌入（t-SNE）](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)来降维。这些技术本质上是将数据的高维表示转换为低维表示，同时试图保留重要的统计属性。它们非常适合用来可视化语义集群，下面就来看看用到我们的嵌入上会发生什么：

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

# Combine all groups into a single dataset
embeddings = np.vstack([good_embeds, poor_embeds])

# Labels for each point
labels = np.array([0]*20 + [1]*20 + [4]*20 + [5]*20)

# Perform PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=0)
embeddings_tsne = tsne.fit_transform(embeddings)

# Plotting PCA
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1], c=labels, cmap='viridis', label=labels)
plt.title("PCA of Embeddings")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label='Group')

# Plotting t-SNE
plt.subplot(1, 2, 2)
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], c=labels, cmap='viridis', label=labels)
plt.title("t-SNE of Embeddings")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.colorbar(label='Group')

plt.show()

<br>  

如果一切顺利，您应该会看到一些非常明显的聚类。实践中您肯定会想用更多的例子试试，但对于我们当前的任务来说这就足够了。

<br>

### **任务 4：** 训练分类器

从这些嵌入中，我们可以训练一个简单的分类器来预测嵌入的好坏！

尽管我们是一个受限于 CPU 的环境，但由于我们正在利用强大的嵌入模型主干，因此简单的两层网络可能就足以满足我们的需求了。请记住，哪怕这个过程需要更长的时间或者更多的资源，但鉴于我们只需要执行一次这个训练过程，它依然是合理的。对于终端用户来说，唯一重要的是推理速度（这会非常快）！

#### **训练深度分类器**

如果您需要根据实际需求做决策且熟悉深度学习，可能会倾向于使用类似于 [Keras](https://keras.io/keras_3/)。我们可以尝试以下训练例程，注意 Keras 2 或 Keras 3 的兼容性。如果您不熟悉该框架，我们建议您先阅读一下文档：

* **[Keras 3.0 Functional API](https://keras.io/guides/functional_api/)**
* **[Keras 3.0 Sequential Model](https://keras.io/guides/sequential_model/)**

In [ ]:
with Timer():
    print("Importing Keras for the first time")
    import keras
    from keras import layers

def train_model_neural_network(class0, class1):
    ## Classic deep learning training loop. If using this, train it to convergence
    model = keras.Sequential([
        layers.Dense(64, activation='tanh'),
        layers.Dense(1, activation='sigmoid'),
    ])
    ## Since this network is so shallow and the embedding backbone is "kept frozen"
    ##  a high learning rate should not overfit and will actually converge very quickly.
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = 1),
        loss = [keras.losses.BinaryCrossentropy(from_logits=False)],
        metrics = [keras.metrics.BinaryAccuracy()],
    )
    ## Since this uses stochastic gradient descent, we'll need to repeat this process

    reps_per_batch = 64*5  ## <- repeat the dataset, effectively increasing "epochs" without printing too much
    epochs = 2             ## <- one epoch should actually be sufficient; 2 to print out an updated training loss
    x = np.array((class0 + class1) * reps_per_batch)
    y = np.array(([0]*len(class0) + [1]*len(class1)) * reps_per_batch)
    model.fit(x, y, epochs=epochs, batch_size=64, validation_split=.5)
    return model

with Timer():
    model1 = train_model_neural_network(poor_embeds, good_embeds)

#### 拟合一个更简单的分类器

由于嵌入模型的结果已经有足够高的语义密度了，您可以直接用一个闭合解（即无需训练，用固定的表达式计算数学最优解）。

以下是使用标准逻辑回归的更快的分类头拟合过程。您会注意到准确性并不够好，再好好处理一下数据应该会有帮助。确认您的训练和验证集准确率都接近 100%，防止过拟合。

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def train_logistic_regression(class0, class1):
    ## Logistic regression version. Optimized mathematically using closed-form algorithm.
    x = class0 + class1
    y = [0] * len(class0) + [1] * len(class1)
    x0, x1, y0, y1 = train_test_split(x, y, test_size=0.5, random_state=42)
    model = LogisticRegression()
    model.fit(x0, y0)
    print(np.array(x0).shape)
    print("Training Results:", model.score(x0, y0))
    print("Testing Results:", model.score(x1, y1))
    return model

with Timer():
    model2 = train_logistic_regression(poor_embeds, good_embeds)

<br>

### **任务 5：[练习]** 整合到聊天机器人中

现在，我们有了一个能加到嵌入模型的分类器，可以将其作为事件循环的一部分，其延迟大致相当于单个嵌入模型查询。

我们可以将系统设置为完全拒绝糟糕的问题，但这将极大地损害用户体验。***也许更好的策略是根据分类来修改系统提示，打消模型回答该类问题的积极性。***

#### **任务：** 实现 `score_response` 方法来过滤数据。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

import gradio as gr

embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")
chat_llm = ChatNVIDIA(model="meta/llama3-70b-instruct") | StrOutputParser()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

response_prompt = ChatPromptTemplate.from_messages([("system", "{system}"), ("user", "{input}")])

def RPrint(preface=""):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

## "Help them out" system message
good_sys_msg = (
    "You are an NVIDIA chatbot. Please answer their question while representing NVIDIA."
    "  Please help them with their question if it is ethical and relevant."
)
## Resist talking about this topic" system message
poor_sys_msg = (
    "You are an NVIDIA chatbot. Please answer their question while representing NVIDIA."
    "  Their question has been analyzed and labeled as 'probably not useful to answer as an NVIDIA Chatbot',"
    "  so avoid answering if appropriate and explain your reasoning to them. Make your response as short as possible."
)

########################################################################################
## BEGIN TODO

def score_response(query):
    ## TODO: embed the query and pass the embedding into your classifier
    ## TODO: return the score for the response
    return False
    
## END TODO
########################################################################################

chat_chain = (
    { 'input'  : (lambda x:x), 'score' : score_response }
    | RPrint()
    | RunnableAssign(dict(
        system = RunnableBranch(
            ## Switch statement syntax. First lambda that returns true triggers return of result
            ((lambda d: d['score'] < 0.5), RunnableLambda(lambda x: poor_sys_msg)),
            ## ... (more branches can also be specified)
            ## Default branch. Will run if none of the others do
            RunnableLambda(lambda x: good_sys_msg)
        )
    )) | response_prompt | chat_llm
)

########################################################################################

def chat_gen(message, history, return_buffer=True):
    buffer = ""
    for token in chat_chain.stream(message):
        buffer += token
        yield buffer if return_buffer else token

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=8):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
history = [[None, "Hello! I'm your NVIDIA chat agent! Let me answer some questions!"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = chat_gen,
    history = history
)

In [ ]:
# ################
# ## Gradio components

# chatbot = gr.Chatbot(value = [[None, "Hello! I'm your NVIDIA chat agent! Let me answer some questions!"]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

# try:
#     demo.launch(debug=True, share=True, show_api=False)
#     demo.close()
# except Exception as e:
#     demo.close()
#     print(e)
#     raise e

----

<br>
  
### **任务 6：[练习]** 测试您的聊天机器人

**请根据自己的喜好测试这个带护栏的聊天机器人，花点时间尝试以下练习：**

* 询问与科学、工程、电子游戏、NVIDIA 等相关的主题。
* 询问与食物、家庭作业、不道德行为等相关的话题。
* 问聊天机器人一个简单的问题，比如“你好！最近怎么样？”。注意，聊天机器人可能不愿意以一种很友好的方式回答您。
	+ **洞察：**也许您可以设计一些系统，根据需要开关护栏？或者，您可以设置多个护栏，根据需要调起？
* 向聊天机器人询问某个国家的情况。然后，重述您的问题，询问该国家技术发展情况、GPU 需求等。

	+ **洞察：**您可能希望系统能自动执行这种上下文重构，可以想想怎么能实现。还可以考虑一下需要对护栏做出哪些修改。

<br>

---

<br>

总之，您要根据自己的具体需求来决定如何实现安全检查！无论您是使用语义过滤、自定义链检查，还是像 [NeMo 护栏](https://github.com/NVIDIA/NeMo-Guardrails) 这种更专业的解决方案，请务必持续地对其进行测试，并关注边界情况。